# The Battle of the Neighborhoods

## 1. Introduction <a name="introduction"></a>

Taking a walk, we may notice that the shops in certain places change frequently. In this case, we can see that stores of the same type are already thriving. In order to prevent the catastrophe of missing some stores of same category, the theme of this project was selected. I am going to make a recommendation system for person who wants to be a shopkeeper. This machine learning model will help people to know what kind of category they should avoid.

## 2. Data <a name="data"></a>

We will use two kinds of data. First, the user's store's latitude and longitude are entered. Second, get store information near your store's location via foursquare api. I am going to set radius as 1000 meter, limit results to 100.

### 2.1. FIrst, get latitude and longitude of the store.

In [1]:
lat, lng = input('input your store\'s latitude and longitude: ').split(',')

input your store's latitude and longitude:  35.704713616784005, 128.45710541327045


### 2.2. Second, using foursquare api, get a dataframe of stores nearby

In [2]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
client_id='JSJ33I0OP3AEKUGF1EHCAZGGZA4CWP5XX012XRMQKSJJLWF4'
client_secret='TPRXXHXBN3IHUGNWRXFYPPWPNFA3ZTT2VFVLFDKGY3CH1SVP'
version='20180605'
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id, 
        client_secret, 
        version, 
        lat, 
        lng, 
        radius, 
        LIMIT)
df = pd.json_normalize(requests.get(url).json()["response"]['groups'][0]['items'])
df.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.location.state,venue.location.address,venue.location.city
0,e-0-56cfa6fbcd106959b101b613-0,0,"[{'summary': 'This spot is popular', 'type': '...",56cfa6fbcd106959b101b613,DGIST 국제관,35.701447,128.460963,"[{'label': 'display', 'lat': 35.701447, 'lng':...",503,KR,대한민국,[대한민국],"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],NaN,NaN,NaN,NaN
1,e-0-56da8a82498eb40ced268d09-1,0,"[{'summary': 'This spot is popular', 'type': '...",56da8a82498eb40ced268d09,EDIYA COFFEE 대구현풍로점,35.698584,128.460960,"[{'label': 'display', 'lat': 35.698584, 'lng':...",766,KR,대한민국,[대한민국],"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN,NaN,NaN,NaN
2,e-0-5b31f8e3180b910039916e01-2,0,"[{'summary': 'This spot is popular', 'type': '...",5b31f8e3180b910039916e01,백세시대,35.701190,128.464612,"[{'label': 'display', 'lat': 35.70119, 'lng': ...",783,KR,대한민국,"[달성군, 대구광역시, 42989, 대한민국]","[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",0,[],42989,대구광역시,NaN,NaN
3,e-0-51f9d246498e7c3a4893fa2e-3,0,"[{'summary': 'This spot is popular', 'type': '...",51f9d246498e7c3a4893fa2e,예家수제순대,35.698571,128.450325,"[{'label': 'display', 'lat': 35.698571, 'lng':...",918,KR,대한민국,"[달성군 현풍면 상리 577-19, 대구광역시, 대구광역시, 대한민국]","[{'id': '56aa371be4b08b9a8d5734e7', 'name': 'G...",0,[],NaN,대구광역시,달성군 현풍면 상리 577-19,대구광역시
4,e-0-5a3f507962420b2b2cd7f865-4,0,"[{'summary': 'This spot is popular', 'type': '...",5a3f507962420b2b2cd7f865,찻집하다(곶감하다),35.700916,128.467030,"[{'label': 'display', 'lat': 35.700916, 'lng':...",991,KR,대한민국,"[달성군 유가면 쌍계리 630-5 1층, 대구광역시, 대구광역시, 711-880, ...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],711-880,대구광역시,달성군 유가면 쌍계리 630-5 1층,대구광역시


## 3. Methodology <a name="methodology"></a>

### 3.1. Preprocess
Sice algorithm only needs location and category, I left latitude, longitude and category data and eraised rest of dataframe.

In [3]:
df = df.loc[:,['venue.location.lat', 'venue.location.lng', 'venue.categories']]
for i in range(len(df)):
    b = dict(df['venue.categories'].iloc[i][0])
    df.loc[i, ['venue.categories']] = b['name']
df.head()

,venue.location.lat,venue.location.lng,venue.categories
0,35.701447,128.460963,Hotel
1,35.698584,128.460960,Café
2,35.701190,128.464612,Korean Restaurant
3,35.698571,128.450325,Gukbap Restaurant
4,35.700916,128.467030,Café


### 3.2. Machine Learning
My goal was to give a list about coategories of store a person should avoid. So i used a k-nearest neighbors algorithm. First, I made a set of categories so I can get max number of K. K starting from 1 until length of set made right before, algorithm will determine what category will that place will fit in. Being classified in a specific category means that even if there is a store in that category, it does not have any advantage.

In [4]:
# get a set of categories
cat = set(df['venue.categories'])

In [5]:
# split dataframe to use as dataset
X=[]
y=[]
for i in range(len(df)):
    X.append([float(df['venue.location.lat'].iloc[i]), float(df['venue.location.lng'].iloc[i])])
    y.append(df['venue.categories'].iloc[i])

In [6]:
print(X[:3])
print(y[:3])

[[35.701447, 128.460963], [35.698584, 128.46096], [35.70119, 128.464612]]
['Hotel', 'Café', 'Korean Restaurant']


In [7]:
# train and get list of categories to avoid
from sklearn.neighbors import KNeighborsClassifier
ans = set()

for i in range(1, len(cat)):
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X, y)
    ans.add(neigh.predict([list((float(lat), float(lng)))])[0])
    del(neigh)

In [8]:
ans

{'Café', 'Hotel'}

## 4. Results <a name="results"></a>

As cell right above, by machine learning user could get list of categories that he/she should avoid.

## 5. Discussion <a name="discussion"></a>

So user who was planning to open the shop at (35.704713616784005, 128.45710541327045), should avoid Café and Hotel.

## 6. Conclusion <a name="conclusion"></a>

I only used KNN and didn't changed any hyperparameter except number of neighbors. If there were datasets that shows such as category of the shop, term of the shop survived, I think it would be able to make a model to predict the income of shop at specific place and shop of specific category.